In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D,MaxPool2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.client import device_lib
import itertools
import os 
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from tensorflow.python.client import device_lib
import sklearn
import wandb
from wandb.keras import WandbCallback
import glob


In [2]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ratkac99 (use `wandb login --relogin` to force relogin)


True

In [4]:
wandb.init(project='InceptionV3', entity="ratkac99",
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.0001,
                     "epochs": 40,
                     "batch_size": 2,
                     "loss_function": "categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "malwareDB"
                 })
config = wandb.config

C:\Users\Kacper\.conda\envs\test\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [68]:
train_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/train'
valid_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/valid'
test_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/test'

#1153,4612

In [69]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2,shuffle=False)


Found 994 images belonging to 5 classes.
Found 141 images belonging to 5 classes.
Found 151 images belonging to 5 classes.


In [70]:
assert train_batches.n == 994
assert valid_batches.n == 141
assert test_batches.n == 151
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 5

In [71]:
cm_plot_labels = ['cerberus','hydra','alien','other','legit']

In [72]:
def calculateF1(cm):
    precision = 0
    recall = 0
    TP = cm[4][4]
    FP = 0
    FN = 0
    TN = 0
    for i in range(0,4):
        FP = FP + cm[i][4]
    for i in range (0,4):
        FN = FN + cm[4][i]
    for i in cm:
        for j in i:
            TN = TN + j
    TN = TN - (FP + FN + TP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    return 2*((precision*recall)/(precision+recall))

In [73]:
inceptionv3 = keras.applications.

In [74]:
model = Sequential()
for layer in mobilenet.layers[:-1]:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False

model.add(Dense(units=5, activation='softmax'))

In [75]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                          
                                                                 
 conv_dw_1_relu (ReLU)       (None, 112, 112, 32)     

In [76]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics="accuracy")

In [77]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=40,verbose=2,callbacks=[WandbCallback(data_type="image", validation_data=valid_batches, labels=cm_plot_labels)])

Epoch 1/40
497/497 - 13s - loss: 1.0746 - accuracy: 0.6449 - val_loss: 0.9298 - val_accuracy: 0.6950 - 13s/epoch - 27ms/step
Epoch 2/40
497/497 - 12s - loss: 0.5928 - accuracy: 0.8058 - val_loss: 0.7198 - val_accuracy: 0.7376 - 12s/epoch - 24ms/step
Epoch 3/40
497/497 - 12s - loss: 0.4828 - accuracy: 0.8380 - val_loss: 0.6399 - val_accuracy: 0.7730 - 12s/epoch - 24ms/step
Epoch 4/40
497/497 - 12s - loss: 0.4307 - accuracy: 0.8602 - val_loss: 0.6321 - val_accuracy: 0.7589 - 12s/epoch - 23ms/step
Epoch 5/40
497/497 - 12s - loss: 0.3861 - accuracy: 0.8712 - val_loss: 0.6159 - val_accuracy: 0.7730 - 12s/epoch - 24ms/step
Epoch 6/40
497/497 - 12s - loss: 0.3541 - accuracy: 0.8753 - val_loss: 0.5188 - val_accuracy: 0.7801 - 12s/epoch - 24ms/step
Epoch 7/40
497/497 - 12s - loss: 0.3367 - accuracy: 0.8893 - val_loss: 0.4856 - val_accuracy: 0.7801 - 12s/epoch - 24ms/step
Epoch 8/40
497/497 - 12s - loss: 0.3133 - accuracy: 0.8944 - val_loss: 0.4983 - val_accuracy: 0.7872 - 12s/epoch - 23ms/step


In [78]:
predictions = model.predict(x=test_batches, verbose=0)

In [79]:
cm = confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=-1))

In [80]:
wandb.log({
            "F1_score" : calculateF1(cm),
            "confusion_matrix" : wandb.sklearn.plot_confusion_matrix(test_batches.classes,
                                                            np.argmax(predictions,axis=-1),
                                                            cm_plot_labels)
            })

In [59]:
wandb.finish()

F1_score,▁
accuracy,▁▂▂▃▃▄▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂
val_accuracy,▄▅▄▂▃▅▃▁▅▃▅▅▅▆▇▃▇▄▄▄▃▅▃▆▆▇▅▇▆▇▄▆▆▇▇▇▇█▇▆
val_loss,▃▂▂▅▅▂▃█▂▄▂▂▂▂▂▃▂▃▃▃▆▂▄▃▂▂▆▂▂▁▄▃▂▂▂▂▂▁▂▂
F1_score,0.57658
accuracy,0.93964
best_epoch,37
best_val_loss,0.76273
epoch,39
